In [3]:
%load_ext autoreload
%autoreload 2
from recsys.model import MultitaskRecommender
from recsys.dataset import NewsDataModule

if "datamodule" not in locals():
    datamodule = NewsDataModule("../data", batch_size=32)
    datamodule.prepare_data()
    datamodule.setup()

model = MultitaskRecommender(768, n_categories=datamodule.train_dataset.max_categories)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
datamodule.setup('test')

In [48]:
from ebrec.utils._constants import DEFAULT_CATEGORY_COL, DEFAULT_CATEGORY_STR_COL

datamodule.train_dataset.articles.cast({DEFAULT_CATEGORY_STR_COL: pl.Categorical}).with_columns(pl.col(DEFAULT_CATEGORY_STR_COL).to_physical().alias(DEFAULT_CATEGORY_COL))

article_id,title,body,subtitle,topics,category_str,tokens,category
i32,str,str,str,list[str],cat,list[f32],u32
3037230,"""Ishockey-spill…","""Ambitionerne o…","""ISHOCKEY: Isho…","[""Kriminalitet"", ""Kendt"", … ""Mindre ulykke""]","""sport""","[0.111132, 0.119653, … -0.008356]",0
3044020,"""Prins Harry tv…","""Den britiske t…","""Hoffet tvang P…","[""Kriminalitet"", ""Kendt"", … ""Personfarlig kriminalitet""]","""underholdning""","[0.113472, 0.097556, … -0.016068]",1
3057622,"""Rådden kørsel …","""Slingrende spr…","""Kan ikke straf…","[""Kriminalitet"", ""Transportmiddel"", ""Bil""]","""nyheder""","[0.090726, 0.125526, … -0.011349]",2
3073151,"""Mærsk-arvinger…","""To oldebørn af…","""FANGET I FLODB…","[""Erhverv"", ""Privat virksomhed"", … ""Rejse""]","""nyheder""","[0.129573, 0.132865, … -0.009751]",2
3193383,"""Skød svigersøn…","""En 44-årig mor…","""44-årig kvinde…","[""Kriminalitet"", ""Personfarlig kriminalitet""]","""krimi""","[0.108794, 0.100466, … 0.000001]",3
3196611,"""Zoo-tårnet 100…","""I mange år var…","""I mange år var…","[""Kultur"", ""Museum og seværdighed""]","""ferie""","[0.072012, 0.091164, … -0.022664]",4
3200325,"""Tævet ihjel på…",""". Skolepige vi…","""Sadomasochisti…","[""Kriminalitet"", ""Livsstil"", … ""Samfund""]","""krimi""","[0.078218, 0.088103, … -0.005774]",3
3200913,"""Denne kæp kan …","""Den usædvanlig…","""Nye spor i den…","[""Kriminalitet"", ""Personfarlig kriminalitet""]","""krimi""","[0.102053, 0.091178, … 0.007148]",3
3209311,"""Morder truer m…","""En morder er i…","""En morder er b…","[""Kriminalitet"", ""Personfarlig kriminalitet""]","""krimi""","[0.090207, 0.10986, … -0.00269]",3


AttributeError: 'PolarsDataFrameWrapper' object has no attribute 'dataframe'

In [50]:
scores = model.predictions[0]
labels = model.labels[0]
from torch.nn import functional as F

F.binary_cross_entropy_with_logits(model.predictions[0], model.labels[0].float())

tensor(0.7424)

In [95]:
print(scores.shape) # (batch_size, candidates)
print(labels.shape) # (batch_size, candidates)

# loss = F.cross_entropy(scores, labels)
# print(loss)

torch.Size([5, 5])
torch.Size([5, 5])


RuntimeError: Expected floating point type for target with class probabilities, got Long

In [56]:
model.labels[0].T

tensor([[1, 0, 0, 0, 0],
        [0, 0, 1, 0, 1],
        [0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0]])